In [ ]:
INSERTS:str = "./../../testymolo/media/old_sql/VIRUS_1-2004-11-04_Inserts_2.sql"
TABLES: str = "./../../testymolo/media/old_sql/VIRUS_1-2004-11-04_Tables.sql"


tables_csv: str = "./../../testymolo/media/tables_csv/"
testydata: str = "./../../testymolo/media/data/"
data_json: str = "./../../testymolo/media/data/data.json"
multifasta: str = "./../../testymolo/media/data/sequences.fasta"

In [ ]:
### caution when not all rows same nbr of col
import re
import os

def custom_csv_parser_to_list(infilepath:str):
    with open(infilepath ,'r') as handle:

        rows:list = []

        for line in handle.readlines():
            line = line.strip(';') # removing terminal ';' 
            values:list = [] # re-initialize value list
            val:str = "" # current parsing value
            inquote:bool = False # re-initialize in quote : False
            quote_char:str = '' # ' or "
            for c in line:
                if(c == ','): # encounter comma
                    if(not inquote): ## and not in quote
                        val = val.strip("'")
                        val = val.strip('"')
                        if(len(val) > 0):
                            values.append(val)
                        else:
                            values.append(None)
                        val = ""
                        continue
                elif(c == '"' or c == "'"): # encounter quote
                    if(inquote): # already in quote 
                        if(quote_char == c):  # encounter ending quote mark
                            inquote = False
                        else: # encounter the other quote mark 
                            pass
                    else: # encounter starting quote mark
                        inquote = True
                        quote_char = c
                elif((c == ' ' or c == '\n') and not inquote):  # get rid of whitespaces
                    continue
                val += c
            if(len(val)>0):
                val = val.strip("'")
                val = val.strip('"')
                values.append(val)

            if(len(values) > 0):
                rows.append(values)
                #print(values)  
        
        print("nbr of col", set([len(row) for row in rows]))
        print("nbr of rows", len(rows))
        return rows

In [ ]:
## Organism
Organism_h: list = ["Tax_id", "Name", "Categorie",
                    "Classe", "Ordre", "Fam", "SsFam", "Genre", "Note_org"]
Organism_p: str = os.path.join(tables_csv, 'Organism.csv')
Organism: list = custom_csv_parser_to_list(Organism_p)
print(Organism_h)
print(Organism[0])


In [ ]:
### CAZy_DB
CAZy_DB_h: list = ["DB_ac", "Protein", "DB_nom", "Organism", "abr", "Tax_id", "EC", "_3D_status",
                   "Length", "Sequence", "DB_note", "Created", "Modified", "PP_status", "Lib_sort"]  # 15
CAZy_DB_p: str = os.path.join(tables_csv, 'CAZy_DB.csv')

CAZy_DB: list = custom_csv_parser_to_list(CAZy_DB_p)
print(CAZy_DB[0])

# test on bio.entrez

In [ ]:
from Bio import Entrez
import json

Entrez.email = "VIMVer@univ-amu.fr"

def get_tax_data(taxid):
    try:
        search = Entrez.efetch(id=taxid, db='taxonomy', retmode='xml')
        return Entrez.read(search)
    except:
        return []

def extract_TaxId(taxon):
    return taxon['TaxId']

def extract_Name(taxon):
    return taxon['ScientificName']

def extract_phylo(taxon):
    return taxon['Lineage']

for result in get_tax_data("11020"):
    taxon = {
        'TaxId':extract_TaxId(result),
        'Name':extract_Name(result),
        'phylogeny':extract_phylo(result)
        }
    taxon_json = json.dumps(taxon)
    print(taxon_json)

In [ ]:
for result in get_tax_data("31631"):
    taxon = {
        'TaxId':extract_TaxId(result),
        'Name':extract_Name(result),
        'phylogeny':extract_phylo(result)
        }
    taxon_json = json.dumps(taxon)
    print(taxon_json)

In [ ]:
Organism_json:str = ""

with open(data_json, 'r') as handle:
    DATA = json.load(handle)
    for org in DATA:
        results = []
        n = 0
        for result in get_tax_data(org):
            taxon = {
                'TaxId':extract_TaxId(result),
                'Name':extract_Name(result),
                'phylogeny':extract_phylo(result)
                }
            taxon_json = json.dumps(taxon)
            results.append(taxon_json)
            print(n, taxon_json)
            n += 1
        if(len(results) > 1):
            input(f"{org}: not found ! ([enter]: to continue)")
        elif(len(results) == 1):
            ans = input(f"{org}: only one found ([enter]: to continue or [else][enter]: to delete)")
            if(ans == ""):
                Organism_json += ","+results[0]+"\n"
        else:
            ans = int(input("which to keep ? ([index][enter]: to select)"))
            while(ans == "" or ans < 0 or ans > len(results)):
                ans = input("plz choose between items above !")
            Organism_json += ","+results[ans]+"\n"

import os
with open(os.path.join(testydata, 'Organism.json'), 'w') as outfile :
    outfile.write('['+Organism_json[1:]+']')


# merge old and new 

In [ ]:
ToDoList:list = []
DoneList:list = []

import os


### INITIATLIZE ToDoList
def initialize():
    ToDoList_handler =  open(os.path.join(testydata, "merge", "ToDoList"), '+w')
    DoneList_handler =  open(os.path.join(testydata, "merge", "DoneList"), 'a')
    for item in CAZy_DB:
        taxid = item[5]
        if(not taxid in ToDoList):
            ToDoList.append(taxid)
    for org in Organism:
        taxid = org[0]
        if(not taxid in ToDoList):
            ToDoList.append(taxid)
    ToDoList_handler.write("\n".join(ToDoList))
    ToDoList_handler.close()
    DoneList_handler.close()

def resume():
    with open(os.path.join(testydata, "merge", "ToDoList"), '+r') as handle:
        ToDoList:list = handle.readlines()

        def extraction(taxid:str) -> list:
            results:list = []
            #1+
            bio_entrez_result = get_tax_data(taxid)
            results.append([record for record in bio_entrez_result])
            #2 
            results.append([org for org in Organism if int(org[0]) == int(taxid)])
            #3+
            results.append([item for item in CAZy_DB if int(item[5]) == int(taxid)])

            return results
                       
        def get_TaxId(results:list) -> list:
            #j = 0
            #answers:list = []
            #print("\tBio.entrez")
            for record in results[0]:
                #answers.append(record['TaxId'])
                return record['TaxId']
                print('\t\t', j, record['TaxId'])
                j += 1
            #print("\tOrganism")
            #for org in results[1]:
            #    answers.append(org[0])
            #    print('\t\t', j, org[0])
            #    j += 1
            #cured_list:list = []
            #print("\tDB.ac")
            #for item in results[2]:
            #    if not item[5] in cured_list:
            #        cured_list.append(item[5])
            #for item in cured_list:
            #    answers.append(item)
            #    print('\t\t', j, item)
            #    j += 1
            #return answers

        def get_Name(results:list) -> list:
            #j = 0
            #answers:list = []
            #print("\tBio.entrez")
            for record in results[0]:
            #    answers.append(record['ScientificName'])
                return record['ScientificName']
            #    print('\t\t', j, record['ScientificName'])
            #    j += 1
            #print("\tOrganism")
            #for org in results[1]:
            #    return org[1]
            #    print('\t\t', j, org[1])
            #    j += 1
            #cured_list:list = []
            #print("\tDB.ac")
            #for item in results[2]:
            #    if not item[3] in cured_list:
            #        cured_list.append(item[3])
            #for item in cured_list:
            #    answers.append(item)
            #    print('\t\t', j, item)
            #    j += 1
            #return answers

        def get_Abr(results:list) -> list:
            #j = 0
            #answers:list = []
            #print("\tOrganism")
            #for org in results[1]:
            #    answers.append(org[-1])
            #    print('\t\t', j, org[-1])
            #    j += 1
            cured_list:list = []
            #print("\tDB.ac")
            for item in results[2]:
                if not item[4] in cured_list:
                    cured_list.append(item[4])
            for item in cured_list:
                return item
            #    answers.append(item)
            #    print('\t\t', j, item)
            #    j += 1
            #return answers
            
        def get_phylogeny(results:list) -> list:
            #j = 0 
            #answers:list = []
            #print("\tBio.entrez")
            for record in results[0]:
                return record['Lineage']
                answers.append(record['Lineage'])
                print('\t\t', j, record['Lineage'])
                j += 1
            #print("\tOrganism")
            #for org in results[1]:
            #    a:str = " ; ".join(org[2:-1])
            #    answers.append(a)
            #    print('\t\t', j, a)
            #    j += 1
            #return answers

        ## question/answer 
        def choose(answers:list, key:str) -> str: 
            def is_all_equals(foo:list):
                if len(foo) < 1 :
                    return True
                bar = str(foo[0])
                for item in foo:
                    if str(item) != bar:
                        return False
                return True
            if is_all_equals(answers):
                ans = input(f"{key}: [enter]: to continue ; [+][enter]: to save progression")
                return answers[0]
                #taxon[key] = answers[0]
            else:
                ans = input(f"{key}: [index][enter] to select")
                if ans == '' :
                    ans = 0
                while( int(ans) < 0 or int(ans) >= len(answers)):
                    ans = int(input(f"{key}: plz select one above !"))
                    return answers[int(ans)]
                    #taxon[key] = answers[j]


        for taxid in ToDoList:
            taxon:dict = {}
            taxid:str.strip() = taxid
            
            results:list = extraction(taxid)
            
            ## choose id 
            #print("Choose id :")
            #answers:list = get_TaxId(results)
            #taxon['TaxId'] = choose(answers,'TaxId')
            taxon['TaxId'] = get_TaxId(results)

            ## choose Name
            #print("Choose Name :")
            #answers:list = get_Name(results)
            #taxon['Name'] = choose(answers, 'Name')
            taxon['Name'] = get_Name(results)

            ## choose Abr
            #print("Choose Abr :")
            #answers:list = get_Abr(results)
            #taxon['abr'] = choose(answers, 'abr')
            taxon['abr'] = get_Abr(results)

            ## choose phylogeny
            #print("Choose phylogeny :")
            #answers:list = get_phylogeny(results)
            #taxon['phylogeny'] = choose(answers, 'phylogeny')
            taxon['phylogeny'] = get_phylogeny(results)

            ### finish taxon
            # 1. delete taxon from todolist
            #with open(os.path.join(testydata, "merge", "ToDoList"), '+w') as handle :
                # if taxid is at index 0 ...
            #    handle.write("".join(ToDoList[1:]))
            #copycat:list = [item for item in ToDoList if item.strip() != taxid.strip()]
            #handle.write("".join(copycat))
            #   -> add into donelist            
            with open(os.path.join(testydata, "merge", "DoneList"), '+a') as DoneList_handler:
                DoneList = DoneList_handler.readlines()
                if not taxid in DoneList :
                    DoneList_handler.write(taxid)
                #print(f"Remaing:{len(copycat)}\tDone:{len(DoneList)}")

            yield taxon
        




In [ ]:
initialize()

In [ ]:
#initialize()
resume()

with open(os.path.join(testydata, 'Organism.json'), 'w') as outfile :

    outfile.write(json.dumps(list(resume())))


# Make a tree

In [75]:
import os, json
import numpy as np

testydata: str = "./../../testymolo/media/data/"
tree1:str = ""
Organism:list = []

forest  = np.zeros((8,8))

with open(os.path.join(testydata, "Organism.temp.json")) as handle:
    Organism:list = json.loads(handle.read())
    print(Organism[0])
    

{'TaxId': '11120', 'Name': 'Infectious bronchitis virus', 'abr': 'AIBV[Beaudette]', 'phylogeny': 'Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Gammacoronavirus; Igacovirus; Avian coronavirus'}


In [111]:

def gimme(A:list, B:list) -> int:
    # give count of differences between to phylogenic paths A and B
    c:int = 0
    a,b = 0,0
    while(a < len(A) and b < len(B)):
        if(A[a] != B[b]):
            c += 1
        a += 1
        b += 1
    return c
            
def cupid(people:list) -> list:
    # people: list of ids
    # found the closest couple within the list
    # recursive: new people(list) n-1, couple formed
    if(len(people) <= 2):
        #end 
        #forme the last couple -> root
        return [people[0], people[1]]
    else:
        #found couple
        N:int = len(people)
        # -> calc matrix of differences of remaining people
        NEO:np.array = neo(np.zeros((N,N)))
        # -> -> forme new couple of the lowest score 
        a,b = morpheus(NEO)
        # -> -> -> new people with couple formed 
        remaining_people:list = []
        for i in range(len(people)):
            if not (i == a or i == b):
                remaining_people.append(people[i])
        remaining_people.append([people[a], people[b]])
        print(people[a], people[b])
        return cupid(remaining_people)

def neo(forest:np.array) -> np.array:
    for i in range(len(forest)):
        for j in range(len(forest)):
            if(i == j or forest[i][j] != 0):
                # ignore diagonal 
                # if value already set (!=0)
                continue
            A = Organism[i]['phylogeny'].split(';')
            A = [item.strip() for item in A]
            B = Organism[j]['phylogeny'].split(';')
            B = [item.strip() for item in B]
            forest[i][j] = gimme(A,B)
            forest[j][i] = gimme(A,B)
    #print(forest)
    return forest

def commun_grounds(A:str, B:str) -> str:
    A:list = [a.strip() for a in A.split(';')]
    B:list = [b.strip() for b in B.split(';')]
    AB:list = []
    i,j = 0,0
    while(A[i] == B[j]):
        AB.append(A[i])
        i += 1 
        j += 1
    return '; '.join(AB)

def morpheus(neo:np.array) -> (int,int) :
    # transfrom neo into smith (matrix diagonal)
    smith = agent_smith(neo)
    lowest = lowerdeck(smith)
    indexes:tuple = firstOccur(smith, lowest)
    print(lowest)
    return indexes

def agent_smith(neo:np.array) -> list:
    N = len(neo)
    smith = []
    for k in range(1,N):
        smith.append([])
        for i in range(k):
            smith[k-1].append(neo[k][i])
    return smith

def lowerdeck(smith:list) -> int:
    lowest = 100
    for i in range(len(smith)):
        for j in range(len(smith[i])):
            lowest = min(smith[i][j], lowest)
    return lowest

def firstOccur(Matrix, value) -> tuple:
    for i in range(len(Matrix)):
        for j in range(len(Matrix[i])):
            if(Matrix[i][j] == value):
                return (i,j)

def rewind(people:list, tree:str="") -> str:
    if(len(people) < 1):
        #if int
        if isinstance(people[0], int):
            work:str = Organism[people[0]]['abr'].replace(' ', '_')
            return '(' + tree + ',' + work + ')'
        #if list
        elif isinstance(people[0], list):
            return rewind(people(0), tree) + ',' + tree
    else:
        txt:str = "("
        for k in people:
            if isinstance(k, int):
                work:str = Organism[k]['abr'].replace(' ', '_')
                txt += work
            elif isinstance(k, list):
                txt += rewind(k, tree)
            txt += ','
        tree = txt[:-1] + ')' + tree            
        
    return tree


In [112]:
import os, json
import numpy as np

def main():
    ## params
    testydata: str = "./../../testymolo/media/data/"
    Organism:list = [] 
    with open(os.path.join(testydata, "Organism.temp.json")) as handle:
        Organism:list = json.loads(handle.read())
    people:list = range(len(Organism))

    #Matrix
    n = len(Organism)
    NEO = np.zeros((n,n))
    NEO = neo(NEO)

    #run cupid
    forest:list = cupid(people)
    print('forest:', forest)
    
    #reconstitute newick ###
    newick:str = rewind(forest)+';'
    print('newick:', newick)
    pass

main()

0.0
5 1
0.0
7 2
1.0
4 3
1.0
[5, 1] 6
1.0
[4, 3] [7, 2]
2.0
[[5, 1], 6] 0
forest: [[[4, 3], [7, 2]], [[[5, 1], 6], 0]]
newick: (((TGV[Purdue],MHV[A59]),(SARS-TOR2,HCoV-229E)),(((PEDV_[CV777],BCoV-ENT),HCoV-OC43),AIBV[Beaudette]));


In [ ]:
#with open(os.path.join(testydata, "people.temp.csv"), 'w') as handle:

n = len(Organism)
NEO = np.zeros((n,n))
NEO = neo(NEO)
print('NEO', NEO)

SMITH = agent_smith(NEO)
print('SMITH')
for line in SMITH:
    print(line)

    #handle.write()

NEO [[0. 3. 2. 3. 3. 2. 3. 2.]
 [3. 0. 2. 1. 3. 2. 0. 1.]
 [2. 2. 0. 2. 1. 1. 2. 2.]
 [3. 1. 2. 0. 3. 2. 1. 1.]
 [3. 3. 1. 3. 0. 1. 3. 2.]
 [2. 2. 1. 2. 1. 0. 2. 2.]
 [3. 0. 2. 1. 3. 2. 0. 1.]
 [2. 1. 2. 1. 2. 2. 1. 0.]]
SMITH
[3.0]
[2.0, 2.0]
[3.0, 1.0, 2.0]
[3.0, 3.0, 1.0, 3.0]
[2.0, 2.0, 1.0, 2.0, 1.0]
[3.0, 0.0, 2.0, 1.0, 3.0, 2.0]
[2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 1.0]


In [ ]:
lowest = np.min(SMITH)
indexes = np.where(SMITH == lowest)
print(*indexes)

[]


/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [ ]:
people:list = [ item for item in Organism]
i = 0
for org in Organism:
    print(str(i) , org['abr'], org['phylogeny'])
    i += 1


0 AIBV[Beaudette] Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Gammacoronavirus; Igacovirus; Avian coronavirus
1 BCoV-ENT Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Betacoronavirus; Embecovirus; Betacoronavirus 1
2 HCoV-229E Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Alphacoronavirus; Duvinacovirus
3 MHV[A59] Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Betacoronavirus; Embecovirus; Murine coronavirus
4 TGV[Purdue] Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Alphacoronavirus; Tegacovirus; Alphacoronavirus 1
5 PEDV [CV777] Viruses; Riboviria; Orthornavirae; Pi